# importing all libraries required

In [1]:
import numpy as np
import pydot
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import sys
import cv2 as cv
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os
from subprocess import check_output
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


# Some more libraries required

In [2]:
from subprocess import check_output

from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# values that remain constant throughout

In [1]:
NUM_CLASSES = 5
# we need images of same size so we convert them into the size
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (HEIGHT, WIDTH, DEPTH)
EPOCHS = 10 # number of epochs to train for, initial learning rate and batch size
INIT_LR = 1e-3
BS = 20 #size of batches
#global variables
ImageNameDataHash = {}
ImageNameDataHashTest = {}
uniquePatientIDList = []

## PROCESSING THE IMAGE

In [4]:
def readTrainData(trainDir):
    global ImageNameDataHash
    # loop over the input images
    images = os.listdir(trainDir)
    print("Number of files in " + trainDir + " is " + str(len(images)))
    for imageFileName in images:
        print(imageFileName)
        if (imageFileName == "TrainLabels1.csv" or imageFileName == "194_right.jpeg"):
            continue
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join( trainDir, imageFileName)
        #print(imageFullPath)
        img = load_img(imageFullPath)
        arr = img_to_array(img)  # Numpy array with shape (233,233,3)
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 < HEIGHT or dim2 < WIDTH or dim3 < DEPTH):
            print("Error image dimensions are less than expected "+str(arr.shape))
        arr = cv.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
        #print(arr.shape) # 128,128,3
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 != HEIGHT or dim2 != WIDTH or dim3 != DEPTH):
            print("Error after resize, image dimensions are not equal to expected "+str(arr.shape))
        #print(type(arr))
        # scale the raw pixel intensities to the range [0, 1] - TBD TEST
        arr = np.array(arr, dtype="float") / 255.0
        imageFileName = imageFileName.replace('.jpeg','')
        ImageNameDataHash[str(imageFileName)] = np.array(arr) 
        print(imageFileName)
    return

--If supplying the test data seperately use this
def readTestData(testDir):
    global ImageNameDataHashTest
    # loop over the input images
    images = os.listdir(testDir)
    print("Number of files in " + testDir + " is " + str(len(images)))
    for imageFileName in images:
        if (imageFileName == "TestLabels.csv"):
            continue
        print(imageFileName)
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join(testDir, imageFileName)
        #print(imageFullPath)
        print(imageFullPath)
        
        img = load_img(imageFullPath)
       
        arr = img_to_array(img)  # Numpy array with shape (233,233,3)
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
 
        if (dim1 < HEIGHT or dim2 < WIDTH or dim3 < DEPTH):
            print("Error image dimensions are less than expected "+str(arr.shape))
        arr = cv.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
        #print(arr.shape) # 128,128,3
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 != HEIGHT or dim2 != WIDTH or dim3 != DEPTH):
            print("Error after resize, image dimensions are not equal to expected "+str(arr.shape))
        #print(type(arr))
        # scale the raw pixel intensities to the range [0, 1] - TBD TEST
        
        arr = np.array(arr, dtype="float") / 255.0
        
        imageFileName = imageFileName.replace('.jpg','')
        print(imageFileName)
         
        
        ImageNameDataHashTest[str(imageFileName)] = np.array(arr) 
        
    return

# Resizing and Reshaping the images

In [6]:
from datetime import datetime
print("Loading images at..."+ str(datetime.now()))
sys.stdout.flush()
readTrainData('TrainSet')
print("Loaded " + str(len(ImageNameDataHash)) + " images at..."+ str(datetime.now())) # 1000

Loading images at...2018-12-16 00:22:18.104053
Number of files in TrainSet is 8406
16679_left.jpeg
16679_left
17750_left.jpeg
17750_left
18378_left.jpeg
18378_left
18233_left.jpeg
18233_left
18355_right.jpeg
18355_right
15088_left.jpeg
15088_left
18861_left.jpeg
18861_left
12503_left.jpeg
12503_left
17734_right.jpeg
17734_right
19467_right.jpeg
19467_right
10125_left.jpeg
10125_left
19059_right.jpeg
19059_right
12042_left.jpeg
12042_left
10363_right.jpeg
10363_right
16728_left.jpeg
16728_left
12831_right.jpeg
12831_right
18759_left.jpeg
18759_left
19474_left.jpeg
19474_left
19293_left.jpeg
19293_left
18827_right.jpeg
18827_right
179_left.jpeg
179_left
15227_left.jpeg
15227_left
1672_right.jpeg
1672_right
11856_left.jpeg
11856_left
1505_left.jpeg
1505_left
14173_right.jpeg
14173_right
19064_right.jpeg
19064_right
16528_left.jpeg
16528_left
17326_left.jpeg
17326_left
11113_left.jpeg
11113_left
16914_right.jpeg
16914_right
11870_right.jpeg
11870_right
19293_right.jpeg
19293_right
10272_ri

13242_right
14501_right.jpeg
14501_right
16907_left.jpeg
16907_left
10705_right.jpeg
10705_right
12651_right.jpeg
12651_right
10848_right.jpeg
10848_right
16927_right.jpeg
16927_right
12878_left.jpeg
12878_left
11949_left.jpeg
11949_left
15144_left.jpeg
15144_left
11391_left.jpeg
11391_left
1810_right.jpeg
1810_right
15520_left.jpeg
15520_left
14144_left.jpeg
14144_left
1480_right.jpeg
1480_right
11750_right.jpeg
11750_right
12751_right.jpeg
12751_right
13353_left.jpeg
13353_left
18277_right.jpeg
18277_right
14449_right.jpeg
14449_right
16590_right.jpeg
16590_right
15868_left.jpeg
15868_left
10515_right.jpeg
10515_right
12714_left.jpeg
12714_left
15674_right.jpeg
15674_right
12712_right.jpeg
12712_right
10137_left.jpeg
10137_left
10150_left.jpeg
10150_left
15320_left.jpeg
15320_left
14256_left.jpeg
14256_left
14304_right.jpeg
14304_right
10251_left.jpeg
10251_left
10078_left.jpeg
10078_left
18158_left.jpeg
18158_left
12233_right.jpeg
12233_right
15879_left.jpeg
15879_left
12602_right.j

15851_right
13912_right.jpeg
13912_right
1419_right.jpeg
1419_right
18080_right.jpeg
18080_right
12329_left.jpeg
12329_left
11818_left.jpeg
11818_left
11366_right.jpeg
11366_right
11738_right.jpeg
11738_right
18970_right.jpeg
18970_right
13570_left.jpeg
13570_left
17173_left.jpeg
17173_left
16497_left.jpeg
16497_left
17438_left.jpeg
17438_left
1835_right.jpeg
1835_right
1880_right.jpeg
1880_right
17473_right.jpeg
17473_right
12817_left.jpeg
12817_left
11992_right.jpeg
11992_right
16149_right.jpeg
16149_right
19448_right.jpeg
19448_right
14504_left.jpeg
14504_left
17050_left.jpeg
17050_left
10022_left.jpeg
10022_left
1330_left.jpeg
1330_left
16171_left.jpeg
16171_left
10059_left.jpeg
10059_left
14191_right.jpeg
14191_right
17089_right.jpeg
17089_right
14620_left.jpeg
14620_left
17153_right.jpeg
17153_right
18113_left.jpeg
18113_left
18709_left.jpeg
18709_left
16565_left.jpeg
16565_left
16534_left.jpeg
16534_left
18048_left.jpeg
18048_left
13326_left.jpeg
13326_left
12316_left.jpeg
12316

14876_right
15000_right.jpeg
15000_right
11618_right.jpeg
11618_right
15751_right.jpeg
15751_right
12974_left.jpeg
12974_left
18696_left.jpeg
18696_left
10489_left.jpeg
10489_left
15791_left.jpeg
15791_left
14880_left.jpeg
14880_left
17649_left.jpeg
17649_left
18929_right.jpeg
18929_right
12903_right.jpeg
12903_right
18130_left.jpeg
18130_left
13431_left.jpeg
13431_left
18039_right.jpeg
18039_right
16474_right.jpeg
16474_right
16697_right.jpeg
16697_right
10475_right.jpeg
10475_right
14198_left.jpeg
14198_left
14986_left.jpeg
14986_left
1431_left.jpeg
1431_left
159_left.jpeg
159_left
18573_left.jpeg
18573_left
11789_left.jpeg
11789_left
10386_left.jpeg
10386_left
19134_left.jpeg
19134_left
11364_right.jpeg
11364_right
15932_left.jpeg
15932_left
13878_left.jpeg
13878_left
14094_left.jpeg
14094_left
14874_right.jpeg
14874_right
13610_right.jpeg
13610_right
14724_right.jpeg
14724_right
18487_right.jpeg
18487_right
13644_left.jpeg
13644_left
15639_left.jpeg
15639_left
10645_right.jpeg
1064

16703_right
14564_right.jpeg
14564_right
17608_right.jpeg
17608_right
1145_left.jpeg
1145_left
11085_left.jpeg
11085_left
10437_right.jpeg
10437_right
10263_right.jpeg
10263_right
19344_left.jpeg
19344_left
18655_left.jpeg
18655_left
1037_left.jpeg
1037_left
17036_left.jpeg
17036_left
14097_right.jpeg
14097_right
14805_right.jpeg
14805_right
17456_right.jpeg
17456_right
16877_left.jpeg
16877_left
13931_left.jpeg
13931_left
1785_left.jpeg
1785_left
19097_left.jpeg
19097_left
16038_right.jpeg
16038_right
18429_left.jpeg
18429_left
17887_left.jpeg
17887_left
11953_left.jpeg
11953_left
13621_left.jpeg
13621_left
12271_right.jpeg
12271_right
16301_right.jpeg
16301_right
15859_left.jpeg
15859_left
15450_right.jpeg
15450_right
17644_left.jpeg
17644_left
16548_right.jpeg
16548_right
11929_right.jpeg
11929_right
17731_left.jpeg
17731_left
14852_right.jpeg
14852_right
19104_right.jpeg
19104_right
16474_left.jpeg
16474_left
14973_left.jpeg
14973_left
16272_right.jpeg
16272_right
1702_left.jpeg
17

...

17302_right
12839_right.jpeg
12839_right
19422_right.jpeg
19422_right
17133_right.jpeg
17133_right
12170_left.jpeg
12170_left
18954_left.jpeg
18954_left
17679_right.jpeg
17679_right
13971_right.jpeg
13971_right
17717_right.jpeg
17717_right
12392_right.jpeg
12392_right
19378_right.jpeg
19378_right
13979_left.jpeg
13979_left
10988_left.jpeg
10988_left
16390_right.jpeg
16390_right
14810_left.jpeg
14810_left
12876_right.jpeg
12876_right
15459_right.jpeg
15459_right
14687_left.jpeg
14687_left
16481_left.jpeg
16481_left
12712_left.jpeg
12712_left
12920_left.jpeg
12920_left
15816_right.jpeg
15816_right
10181_left.jpeg
10181_left
16178_right.jpeg
16178_right
14299_right.jpeg
14299_right
16528_right.jpeg
16528_right
14511_left.jpeg
14511_left
14375_left.jpeg
14375_left
10511_right.jpeg
10511_right
10104_left.jpeg
10104_left
1600_right.jpeg
1600_right
16623_right.jpeg
16623_right
12915_right.jpeg
12915_right
12056_right.jpeg
12056_right
12458_left.jpeg
12458_left
1551_left.jpeg
1551_left
17600_r

19168_right
11117_left.jpeg
11117_left
19444_left.jpeg
19444_left
17986_right.jpeg
17986_right
1632_left.jpeg
1632_left
12590_left.jpeg
12590_left
12469_right.jpeg
12469_right
10147_right.jpeg
10147_right
17872_right.jpeg
17872_right
18163_left.jpeg
18163_left
12778_left.jpeg
12778_left
16616_left.jpeg
16616_left
10643_left.jpeg
10643_left
17699_right.jpeg
17699_right
18786_right.jpeg
18786_right
11896_right.jpeg
11896_right
11952_right.jpeg
11952_right
10886_left.jpeg
10886_left
17009_left.jpeg
17009_left
11005_right.jpeg
11005_right
10199_left.jpeg
10199_left
11407_left.jpeg
11407_left
16692_left.jpeg
16692_left
16021_right.jpeg
16021_right
19406_left.jpeg
19406_left
17357_left.jpeg
17357_left
19329_left.jpeg
19329_left
14856_left.jpeg
14856_left
10262_right.jpeg
10262_right
18364_right.jpeg
18364_right
18862_left.jpeg
18862_left
1076_right.jpeg
1076_right
19171_left.jpeg
19171_left
18016_left.jpeg
18016_left
10558_left.jpeg
10558_left
14913_right.jpeg
14913_right
16558_left.jpeg
165

17548_left
10680_left.jpeg
10680_left
17766_right.jpeg
17766_right
10664_right.jpeg
10664_right
14361_left.jpeg
14361_left
19043_left.jpeg
19043_left
17838_right.jpeg
17838_right
11923_left.jpeg
11923_left
15227_right.jpeg
15227_right
1878_right.jpeg
1878_right
18107_left.jpeg
18107_left
16815_right.jpeg
16815_right
12133_right.jpeg
12133_right
1387_left.jpeg
1387_left
14205_left.jpeg
14205_left
13888_right.jpeg
13888_right
15118_left.jpeg
15118_left
12070_right.jpeg
12070_right
16379_right.jpeg
16379_right
10421_right.jpeg
10421_right
14115_right.jpeg
14115_right
17645_right.jpeg
17645_right
1592_right.jpeg
1592_right
13627_right.jpeg
13627_right
12110_left.jpeg
12110_left
12951_right.jpeg
12951_right
17953_right.jpeg
17953_right
12724_left.jpeg
12724_left
12413_left.jpeg
12413_left
18656_right.jpeg
18656_right
11304_right.jpeg
11304_right
12975_left.jpeg
12975_left
10701_left.jpeg
10701_left
12299_left.jpeg
12299_left
15431_left.jpeg
15431_left
15336_left.jpeg
15336_left
11830_right.

15973_left
15039_left.jpeg
15039_left
18798_right.jpeg
18798_right
1802_right.jpeg
1802_right
14533_left.jpeg
14533_left
18000_right.jpeg
18000_right
17722_left.jpeg
17722_left
12445_left.jpeg
12445_left
12779_right.jpeg
12779_right
1029_left.jpeg
1029_left
17037_right.jpeg
17037_right
18005_right.jpeg
18005_right
10646_right.jpeg
10646_right
12206_left.jpeg
12206_left
18337_right.jpeg
18337_right
12205_right.jpeg
12205_right
10194_right.jpeg
10194_right
14388_right.jpeg
14388_right
12958_right.jpeg
12958_right
15821_right.jpeg
15821_right
15781_right.jpeg
15781_right
16842_right.jpeg
16842_right
1803_right.jpeg
1803_right
1307_right.jpeg
1307_right
16191_left.jpeg
16191_left
17487_left.jpeg
17487_left
13602_left.jpeg
13602_left
16763_left.jpeg
16763_left
16114_left.jpeg
16114_left
1847_right.jpeg
1847_right
12521_left.jpeg
12521_left
13790_left.jpeg
13790_left
15376_right.jpeg
15376_right
19270_left.jpeg
19270_left
14219_right.jpeg
14219_right
1351_left.jpeg
1351_left
11758_right.jpeg

14806_right
11159_right.jpeg
11159_right
11741_right.jpeg
11741_right
10438_left.jpeg
10438_left
15255_left.jpeg
15255_left
13848_left.jpeg
13848_left
14431_left.jpeg
14431_left
13776_right.jpeg
13776_right
13099_left.jpeg
13099_left
10727_left.jpeg
10727_left
13815_right.jpeg
13815_right
12904_left.jpeg
12904_left
17604_left.jpeg
17604_left
19083_right.jpeg
19083_right
1785_right.jpeg
1785_right
17882_right.jpeg
17882_right
12620_left.jpeg
12620_left
14904_right.jpeg
14904_right
18038_left.jpeg
18038_left
16862_right.jpeg
16862_right
15438_left.jpeg
15438_left
16419_left.jpeg
16419_left
157_left.jpeg
157_left
11163_left.jpeg
11163_left
14311_left.jpeg
14311_left
16673_left.jpeg
16673_left
15175_right.jpeg
15175_right
16707_left.jpeg
16707_left
1113_left.jpeg
1113_left
11198_left.jpeg
11198_left
10907_right.jpeg
10907_right
18257_right.jpeg
18257_right
13994_left.jpeg
13994_left
18010_right.jpeg
18010_right
14534_right.jpeg
14534_right
12759_right.jpeg
12759_right
18078_left.jpeg
18078

# Linking individual images with their respective data 

In [7]:
import csv
def readTrainCsv():
    raw_df = pd.read_csv('B. Disease Grading/1. Original-Images/TrainSet/TrainLabels.csv', sep=',')
    print(type(raw_df)) #<class 'pandas.core.frame.DataFrame'>
    row_count=raw_df.shape[0] #gives number of row count row_count=35126 
    col_count=raw_df.shape[1] #gives number of col count col count=2
    print("row_count="+str(row_count)+" col count="+str(col_count))
    raw_df["PatientID"] = ''
    header_list = list(raw_df.columns)
    print(header_list) # ['image', 'level', 'PatientID']
    # double check if level of left and right are same or not
    ImageLevelHash = {}
    patientIDList = []
    for index, row in raw_df.iterrows():
        # 0 is image, 1 is level, 2 is PatientID, 3 is data
        key = row[0] + ''
        patientID = row[0] + ''
        patientID = patientID.replace('_right','')
        patientID = patientID.replace('_left','')
        #print("Adding patient ID"+ patientID)
        raw_df.at[index, 'PatientID'] = patientID
        patientIDList.append(patientID)
        ImageLevelHash[key] = str(row[1]) # level
                
    global uniquePatientIDList
    uniquePatientIDList = sorted(set(patientIDList))
    count=0;
    for patientID in uniquePatientIDList:
        left_level = ImageLevelHash[str(patientID+'_left')]
        right_level = ImageLevelHash[str(patientID+'_right')]
        #right_exists = str(patientID+'_right') in raw_df.values
        if (left_level != right_level):
            count = count+1
            #print("Warning for patient="+ str(patientID) + " left_level=" + left_level+ " right_level=" +right_level)
    print("count of images with both left and right eye level not matching="+str(count)) # 2240
    print("number of unique patients="+str(len(uniquePatientIDList))) # 17563
    return raw_df

In [8]:
raw_df = pd.read_csv('TrainSet/TrainLabels1.csv', sep=',',skip_blank_lines=True,date_parser="Image name")
raw_df=raw_df.loc[:, ~raw_df.columns.str.contains('^Unnamed')]
#raw_df_test=pd.read_csv('B. Disease Grading/1. Original-Images/TestSet/TestLabels.csv', sep=',',skip_blank_lines=True,date_parser="Image name")
#raw_df_test=raw_df_test.loc[:, ~raw_df_test.columns.str.contains('^Unnamed')]
rawddf_num = np.array(raw_df)
#rawddf_num[:,:1]
print(len(rawddf_num[:,:1]))
#rawddf_num[:,1:]
#raw_df_test_num=np.array(raw_df_test)
#raw_df_test_num[0][1]

8405


In [9]:
len(rawddf_num[:,:1])

8405

In [10]:
len(ImageNameDataHash)

8405

In [11]:
header_list = list(raw_df.columns)
header_list

['image', 'level']

# Creating a matrix with coloums containging image name, array, Retinopathy grade

In [12]:
imageNameArr = []
dataArr = []
rgradearr= []

num=len(rawddf_num[:,:1])
num=num
for index, row in raw_df.iterrows():
    key = str(row[0])
    if key in ImageNameDataHash:
        imageNameArr.append(key)
        dataArr.append(np.array(ImageNameDataHash[key])) # np.array
for i in range(num) :
    rgradearr.append(rawddf_num[i][1])
    i=i+1
print(len(rgradearr))
df2 = pd.DataFrame({'Image_name': imageNameArr, 'data': dataArr, 'Retinopathy Grade': rgradearr})
df2_header_list = list(df2.columns) 
print(df2_header_list) # ['image', 'data']
print(len(df2)) # 1000


8405
['Image_name', 'data', 'Retinopathy Grade']
8405


In [13]:
X = df2['data']
Y = df2['Retinopathy Grade']
Y = np.array(Y)
Y =  to_categorical(Y, num_classes=NUM_CLASSES)


# Dividing the data for testing and training purposes

In [14]:
print("Parttition data into 80:20...")
sys.stdout.flush()
unique_ids = df2.Image_name.unique()
unique_ids
#unique_idstest = df2test.Image_name.unique()
train_ids, valid_ids = train_test_split(unique_ids, test_size = 0.2, random_state = 10)
#train_ids=unique_ids
#valid_ids=unique_idstest
trainid_list = train_ids.tolist()
validid_list = valid_ids.tolist()
print('trainid_list shape=', str(len(trainid_list))) # 375
print('testid_list shape=', str(len(validid_list)))
traindf = df2[df2.Image_name.isin(trainid_list)]
valSet = df2[~df2.Image_name.isin(trainid_list)]

Parttition data into 80:20...
trainid_list shape= 6724
testid_list shape= 1681


In [15]:


print(traindf.head())
print(valSet.head())

traindf = traindf.reset_index(drop=True)
valSet = valSet.reset_index(drop=True)

print(traindf.head())
print(valSet.head())



  Image_name                                               data  \
0    10_left  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
1   10_right  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
2    13_left  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
3   13_right  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
4    15_left  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   

   Retinopathy Grade  
0                  0  
1                  0  
2                  0  
3                  0  
4                  1  
   Image_name                                               data  \
5    15_right  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
8     17_left  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
16   108_left  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
17  108_right  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   
21  114_right  [[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,...   

    Retinopathy Grade  
5                   2  
8 

In [16]:
trainX = traindf['data']
trainY = traindf['Retinopathy Grade']

valX = valSet['data']
valY = valSet['Retinopathy Grade']

#(trainX, valX, trainY, valY) = train_test_split(X,Y,test_size=0.20, random_state=10)
print('trainX shape=', trainX.shape[0], 'valX shape=', valX.shape[0])

trainX shape= 6724 valX shape= 1681


In [18]:
trainY =  to_categorical(trainY, num_classes=NUM_CLASSES)
valY =  to_categorical(valY, num_classes=NUM_CLASSES)

In [19]:
print("Generating images...")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")

Generating images...


# Main Model: Using CNNs, MaxPooling and taking the output

In [20]:
def createModel():
    model = Sequential()
    # first set of CONV => RELU => MAX POOL layers
    model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=inputShape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim=NUM_CLASSES, activation='softmax'))
    # returns our fully constructed deep learning + Keras image classifier 
    opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    # use binary_crossentropy if there are two classes
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model


# Reshaping the Array

In [22]:
print("Reshaping trainX at..."+ str(datetime.now()))
#print(trainX.sample()) 
print(type(trainX)) # <class 'pandas.core.series.Series'>
print(trainX.shape) # (750,)
from numpy import zeros

Xtrain = np.zeros([trainX.shape[0],128,128,3])

for i in range(trainX.shape[0]): # 0 to traindf Size -1
    Xtrain[i] = trainX[i]
print("yoooo")
print(Xtrain.shape) # (750,128,128,3)""
print("Reshaped trainX at..."+ str(datetime.now()))

Reshaping trainX at...2018-12-16 00:44:35.876412
<class 'pandas.core.series.Series'>
(6724,)
yoooo
(6724, 128, 128, 3)
Reshaped trainX at...2018-12-16 00:45:42.612978


In [23]:
print("Reshaping valX at..."+ str(datetime.now()))
print(type(valX)) # <class 'pandas.core.series.Series'>
print(valX.shape) # (250,)
from numpy import zeros
sys.stdout.flush()
Xval = np.zeros([valX.shape[0],HEIGHT, WIDTH, DEPTH])
for i in range(valX.shape[0]): # 0 to traindf Size -1
    Xval[i] = valX[i]
print(Xval.shape) # (250,128,128,3)
print("Reshaped valX at..."+ str(datetime.now()))

Reshaping valX at...2018-12-16 00:46:00.125312
<class 'pandas.core.series.Series'>
(1681,)
(1681, 128, 128, 3)
Reshaped valX at...2018-12-16 00:46:16.616483


In [24]:
print("compiling model...")
sys.stdout.flush()
model = createModel()

# print the summary of model
from keras.utils import print_summary
print_summary(model, line_length=None, positions=None, print_fn=None)

# add some visualization
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
SVG(model_to_dot(model).create(prog='dot', format='svg'))

compiling model...


/home/nemathahmed/.local/lib/python3.6/site-packages/ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=5)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 63, 63, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 30, 64)        0         
__________

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

# Training and saving the model

In [2]:
# train the network
print("training network...")
sys.stdout.flush()
#class_mode ='categorical', # 2D one-hot encoded labels
H = model.fit_generator(aug.flow(Xtrain, trainY, batch_size=BS), \
    validation_data=(Xval, valY), \
    steps_per_epoch=len(trainX) // BS, \
    epochs=4, verbose=1) 

# save the model to disk
print("Saving model to disk")
sys.stdout.flush()
model.save("/tmp/mymodel2")
model.save("dd2.model")

training network...


NameError: name 'sys' is not defined

In [ ]:
print("Generating plots...")
import matplotlib
sys.stdout.flush()
matplotlib.use("Agg")
matplotlib.pyplot.style.use("ggplot")
matplotlib.pyplot.figure()
N = EPOCHS
matplotlib.pyplot.plot(np.arange(0, N), H.history["loss"], label="train_loss")
matplotlib.pyplot.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
matplotlib.pyplot.plot(np.arange(0, N), H.history["acc"], label="train_acc")
matplotlib.pyplot.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
matplotlib.pyplot.title("Training Loss and Accuracy on diabetic retinopathy detection")
matplotlib.pyplot.xlabel("Epoch #")
matplotlib.pyplot.ylabel("Loss/Accuracy")
matplotlib.pyplot.legend(loc="lower left")
matplotlib.pyplot.savefig("plot.png")

In [5]:
import tensorflow as tf
from numpy import zeros
img21 = load_img("TrainSet/10_right.jpeg")
arr21 = img_to_array(img21)
arr21 = cv.resize(arr21, (HEIGHT,WIDTH))
arr21 = np.array(arr21, dtype="float") / 255.0

sys.stdout.flush()
Tval = np.zeros([arr21.shape[0],HEIGHT, WIDTH, DEPTH])
 # 0 to traindf Size -1
#Tval = arr
print(Tval.shape) # (250,128,128,3)

m21=arr21.reshape(1,128,128,3)
for m in range(arr21.shape[0]): # 0 to traindf Size -1
    Tval[m] = arr21[m]
print(Tval.shape)
model = tf.keras.models.load_model("dd.model")
prediction=model.predict([m21])
print(prediction)

NameError: name 'load_img' is not defined

 def readTrainData(trainDir):
    global ImageNameDataHash
    # loop over the input images
    images = os.listdir(trainDir)
    print("Number of files in " + trainDir + " is " + str(len(images)))
    for imageFileName in images:
        print(imageFileName)
        if (imageFileName == "TrainLabels1.csv" or imageFileName == "194_right.jpeg"):
            continue
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join( trainDir, imageFileName)
        #print(imageFullPath)
        img = load_img(imageFullPath)
        arr = img_to_array(img)  # Numpy array with shape (233,233,3)
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 < HEIGHT or dim2 < WIDTH or dim3 < DEPTH):
            print("Error image dimensions are less than expected "+str(arr.shape))
        arr = cv.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
        #print(arr.shape) # 128,128,3
        dim1 = arr.shape[0]
        dim2 = arr.shape[1]
        dim3 = arr.shape[2]
        if (dim1 != HEIGHT or dim2 != WIDTH or dim3 != DEPTH):
            print("Error after resize, image dimensions are not equal to expected "+str(arr.shape))
        #print(type(arr))
        # scale the raw pixel intensities to the range [0, 1] - TBD TEST
        arr = np.array(arr, dtype="float") / 255.0
        imageFileName = imageFileName.replace('.jpeg','')
        ImageNameDataHash[str(imageFileName)] = np.array(arr) 
        print(imageFileName)
    return

















